## dlt Workshop
**This is the homework URL of Data Engineering Zoomcamp 2024 - dlt workshop.**
<br>Course link: https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/cohorts/2024/workshops/dlt.md
<br>Homework link: https://colab.research.google.com/drive/1Te-AT0lfh0GpChg1Rbd0ByEKOHYtWXfm#scrollTo=vjWhILzGJMpK&forceEdit=true&sandboxMode=true

### Question 1:
Here we need to return the generator output as a list to retrieve all values.

In [9]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = [sqrt_value for sqrt_value in square_root_generator(limit)]
print(generator)
sum_output = sum(generator)
print(sum_output)

[1.0, 1.4142135623730951, 1.7320508075688772, 2.0, 2.23606797749979]
8.382332347441762


### Question 2:
Here we set `limit=13` and get last value from the list

In [10]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = [sqrt_value for sqrt_value in square_root_generator(limit)]
print(generator[-1])

3.605551275463989


### Question 3:
1. Load people_1 to table `people`
2. Use duckdb to query age sum in table `people`
3. Append people_2 to table `people`, note `write_disposition=append`
4. Use duckdb to query age sum in table `people` (for both people_1 and people_2)

In [3]:
%%capture
!pip3 install dlt[duckdb]

In [4]:
import dlt
import duckdb

# generate people_1
def people_1():
    for i in range(1, 6):
       yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# load people_1 to people table
people_generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
load_people_1 = people_generators_pipeline.run(people_1(), table_name="people", write_disposition="replace")

conn = duckdb.connect(f"{people_generators_pipeline.pipeline_name}.duckdb")

query_people = conn.sql("SELECT * FROM generators.people").df()
print('Get people_1 data:')
display(query_people)

# get age sum of people_1
query_people_1_sum = conn.sql("SELECT sum(age) FROM generators.people").df()
print('Get age sum of people_1:')
display(query_people_1_sum)

# generate people_2
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# append people_2 to people table
load_people_2 = people_generators_pipeline.run(people_2(), table_name="people", write_disposition="append")

query_people = conn.sql("SELECT * FROM generators.people").df()
print('Get both people_1 and people_2 data:')
display(query_people)

query_people_sum = conn.sql("SELECT sum(age) FROM generators.people").df()
print('Get age sum of all people:')
display(query_people_sum)

Get people_1 data:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708268989.9422815,AjwN9aUjL7+22Q,None
1,2,Person_2,27,City_A,1708268989.9422815,d+7Ijr1oKl4F1w,None
2,3,Person_3,28,City_A,1708268989.9422815,fzapnxvF5WTYfQ,None
3,4,Person_4,29,City_A,1708268989.9422815,twSD0yHGv6Tn4w,None
4,5,Person_5,30,City_A,1708268989.9422815,NsLOTucGrBDcng,None


Get age sum of people_1:


,sum(age)
0,140.0


Get both people_1 and people_2 data:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708268989.9422815,AjwN9aUjL7+22Q,None
1,2,Person_2,27,City_A,1708268989.9422815,d+7Ijr1oKl4F1w,None
2,3,Person_3,28,City_A,1708268989.9422815,fzapnxvF5WTYfQ,None
3,4,Person_4,29,City_A,1708268989.9422815,twSD0yHGv6Tn4w,None
4,5,Person_5,30,City_A,1708268989.9422815,NsLOTucGrBDcng,None
5,3,Person_3,33,City_B,1708268990.5862682,21laSSKGREX6oA,Job_3
6,4,Person_4,34,City_B,1708268990.5862682,tF1kM6pPRKP8OQ,Job_4
7,5,Person_5,35,City_B,1708268990.5862682,wdsPMA+s2FSKQw,Job_5
8,6,Person_6,36,City_B,1708268990.5862682,Sv4JJzwpCVBUfw,Job_6
9,7,Person_7,37,City_B,1708268990.5862682,STWUi/luTf3LWQ,Job_7


Get age sum of all people:


,sum(age)
0,353.0


### Question 4:
Answer is similar to Question 3, except for people_2 we change the `write_disposition` to `merge`
<br>
```
load_people_2 = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="merge", 
                                        primary_key='id')
```

In [5]:
import dlt
import duckdb

# generate people_1
def people_1():
    for i in range(1, 6):
       yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# load people_1 to people table
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
load_people_1 = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

query_people = conn.sql("SELECT * FROM generators.people").df()
print('Get people_1 data:')
display(query_people)

# get age sum of people_1
query_people_1_sum = conn.sql("SELECT sum(age) FROM generators.people").df()
print('Get age sum of people_1:')
display(query_people_1_sum)

# generate people_2
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# append people_2 to people table
load_people_2 = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="merge",
                                        primary_key='id')

query_people = conn.sql("SELECT * FROM generators.people").df()
print('Get both people_1 and people_2 data:')
display(query_people)

query_people_sum = conn.sql("SELECT sum(age) FROM generators.people").df()
print('Get age sum of all people:')
display(query_people_sum)

Get people_1 data:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708269001.2756596,QRLo39ZRMc9QCg,None
1,2,Person_2,27,City_A,1708269001.2756596,iyOH0y5BO8rVxQ,None
2,3,Person_3,28,City_A,1708269001.2756596,ITCbpjjVlx1u0g,None
3,4,Person_4,29,City_A,1708269001.2756596,rQWg+MsS12T/Zg,None
4,5,Person_5,30,City_A,1708269001.2756596,4kjTPrc2DcDZpQ,None


Get age sum of people_1:


,sum(age)
0,140.0


Get both people_1 and people_2 data:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708269001.2756596,QRLo39ZRMc9QCg,None
1,2,Person_2,27,City_A,1708269001.2756596,iyOH0y5BO8rVxQ,None
2,8,Person_8,38,City_B,1708269001.9179623,TETxDqaybjGH0w,Job_8
3,7,Person_7,37,City_B,1708269001.9179623,c3bfgJmNC8KSKQ,Job_7
4,5,Person_5,35,City_B,1708269001.9179623,4kQ4drgR8wDxiA,Job_5
5,3,Person_3,33,City_B,1708269001.9179623,KyyUmVxxX/tFVg,Job_3
6,6,Person_6,36,City_B,1708269001.9179623,2buoUChoDTVx6A,Job_6
7,4,Person_4,34,City_B,1708269001.9179623,JoTBC+TGWfBCPA,Job_4


Get age sum of all people:


,sum(age)
0,266.0
